# K Nearest Neighbors

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import numpy as np
from sklearn import preprocessing, neighbors
from sklearn. model_selection import cross_val_score
import pandas as pd
from sklearn.model_selection import train_test_split

In [72]:
df = pd.read_csv("/content/drive/MyDrive/Python Machine Learning JGG/datasets/cancer/breast-cancer-wisconsin.data.txt", header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [73]:
df[1]   ##columna primera de datos

0      5
1      5
2      3
3      6
4      4
      ..
694    3
695    2
696    5
697    4
698    4
Name: 1, Length: 699, dtype: int64

In [74]:
df.shape

(699, 11)

#  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)

In [75]:
df.describe()

,0,1,2,3,4,5,7,8,9,10
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [ ]:
# si tenemos algún dato que falta, pondríamos un valor muy muy negativo (-1000000) para que no afecte a la distancia

In [76]:
# los nombres de las columnas deberían ser nombres puestos por nosotros

df.columns = ["name", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "class"]

In [77]:
df.head()

,name,V1,V2,V3,V4,V5,V6,V7,V8,V9,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [ ]:
# no nos hace falta la primera columna, la eliminamos

In [78]:
df.drop(["name"],axis=1)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2,1,1,1,2
695,2,1,1,1,2,1,1,1,1,2
696,5,10,10,3,7,3,8,10,2,4
697,4,8,6,4,3,4,10,6,1,4


In [79]:
df.replace("?",-999999, inplace=True)   #True para que haga el reemplazo directamente

In [80]:
# si quiero tener separado los elementos clasficantes y la que quiero usar para clasificar:

Y = df["class"]
X = df[["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"]]

In [81]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1
3,6,8,8,1,3,4,3,7,1
4,4,1,1,3,2,1,3,1,1


In [82]:
Y.head()

0    2
1    2
2    2
3    2
4    2
Name: class, dtype: int64

## Clasificador de los K vecinos

In [83]:
# vamos a entrenar el algoritmo: usamos la validacion cruzada
# separamos en una parte para entrenar y para test tanto para x como para y

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [84]:
clf = neighbors.KNeighborsClassifier()

In [85]:
clf.fit(X_train, Y_train)

KNeighborsClassifier()

In [86]:
accuracy = clf.score(X_test, Y_test)
accuracy

0.9928571428571429

In [87]:
# el test ha salido bien, es elevado

In [88]:
accuracy_train = clf.score(X_train, Y_train)
accuracy_train

0.9749552772808586

In [ ]:
# veamos cómo saldría el resultado si no limpiamos los datos de la primera columna, la que era el id de los pacientes y que era un numero muy alto:

# Clasificación sin limpieza

In [96]:
df2 = pd.read_csv("/content/drive/MyDrive/Python Machine Learning JGG/datasets/cancer/breast-cancer-wisconsin.data.txt", header=None)
df2.replace("?", -99999, inplace=True)
df2.columns = ["name", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "class"]

Y2 = df["class"]
X2 = df[["name", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"]]

X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.2)

clf2 = neighbors.KNeighborsClassifier()
clf2.fit(X2_train, Y2_train)

accuracy2 = clf2.score(X2_test, Y2_test)
accuracy2

0.6142857142857143

In [ ]:
# da un resultado muy malo!!!!

In [ ]:
# veremos como usar este algoritmo para clasificar una nueva muestra de datos

# Clasificar nuevos datos

In [97]:
# imaginamos que recibimos estos datos de un paciente nuevo:

sample_measure = np.array([4,2,1,1,1,2,3,2,1])

In [98]:
sample_measure = sample_measure.reshape(1,-1)

In [99]:
predict = clf.predict(sample_measure)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [100]:
predict

array([2])

In [ ]:
# esto me dice que ha predicho que la clase es 2: benigno

In [ ]:
# si decido hacer los test dos veces porque soy hipocondricaco:

In [101]:
sample_measure2 = np.array([[4,2,1,1,1,2,3,2,1], [4,2,1,1,1,2,3,2,1]]).reshape(2,-1)

In [102]:
predict = clf.predict(sample_measure2)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [103]:
predict

array([2, 2])